<a href="https://colab.research.google.com/github/Valerik-Bogd/Punctuation/blob/main/Prepare_datasets_distilled_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# name of the general folder
data_path = '/content/drive/MyDrive/Data_punct'

# raw_data_path = '/content/drive/MyDrive/Data_punct/Raw_data'
# out_path = '/content/drive/MyDrive/Data_punct/Datasets'

# name of folder to prepare, which lies in raw_data_path:
data_to_conv = 'Legal_docx'
files_ext = 'txt'

In [ ]:
data_to_conv_path = f'{data_path}/Raw_data/{data_to_conv}'

In [ ]:
# imports
from google.colab import drive
drive.mount('/content/drive')

import glob
import os
import csv


import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import pandas as pd

!pip install pypandoc
import pypandoc

!pip install docx2txt
import docx2txt
from tqdm import tqdm


Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=2d86aa89f3449cddef2441f32fb3047f70b0bece89a4b981d079fc6a8d8d43cc
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
class Files_Prep:
    def __init__(self, data_to_conv, files_ext, data_path='/content/drive/MyDrive/Data_punct'):
        self.data_to_conv_path = f'{data_path}/Raw_data/{data_to_conv}'
        self.files_ext = files_ext
        self.delete_list = [" Документ предоставлен ", " КонсультантПлюс: ", " КонсультантПлюс ",
                            " [] ", " []\n", "\n[] ", "\r[]\r", "+-", " -- ", " -+", "+=","=+",
                            " = ", " | ", " + ", " -", "\r\n []", "\n []", "\n\n", "\n[]\n", "$[]$"]

        self.all_punct = ["'", '"',".", ",", "!", "?","-", ":", ";"]
        self.names_punct = ['S_QUOTE', 'D_QUOTE',
                            'PERIOD', 'COMMA', 'EX_MARK', 'Q_MARK',
                            'DASH', 'COLUMN', 'SEMI_COL'
                            ]
        self.punct_marks = dict(zip(self.all_punct, self.names_punct))

        self.files = []
        # r=root, d=directories, f = files
        for r, d, f in os.walk(self.data_to_conv_path):
            for file in f:
                if file.endswith((f'.{files_ext}')):
                    self.files.append(os.path.join(r, file))
        print(self.files)

    def process_files(self):
        print(self.files_ext)
        # files = os.listdir(self.data_to_conv_path)

        if self.files_ext not in ('txt', 'csv'):
            self.convert_to_txt(self.files)

        if self.files_ext == 'txt':
            self.clean_split_txt(self.files)
        elif self.files_ext == 'csv':
            self.clean_split_csv(self.files)

        # now files ought to be .txt paragraphs in dataset folder
        self.make_to_dataset(f'{data_path}/Datasets/{data_to_conv}')


    def convert_to_txt(self, file_path):
        # Convert the file to txt format
        if self.files_ext not in ('doc', 'docx'):
            print('not ready')
            return

        elif self.files_ext in ('doc', 'docx'):
            self.txt_files = []
            for doc_file_name in self.files:
                file_name = doc_file_name.split("/")[-1]
                txtFilename = data_path + "/Raw_data/" + data_to_conv + "/" + file_name.split(".")[0] + ".txt"
                self.txt_files.append(txtFilename)
                print ('files will be saved as:', txtFilename)
                #output = pypandoc.convert_file(docxFilename, 'plain', txtFilename)
                output = pypandoc.convert_file(doc_file_name, 'plain',extra_args=('--standalone','--wrap=none'), outputfile=txtFilename)
                assert output == ""
                self.files_ext = 'txt'
            return
            # return self.txt_files


    def clean_split_txt(self, file_path):
        # Delete junk from a txt file
        counter = 0

        if not os.path.exists(f"{data_path}/Datasets/{data_to_conv}"):
            os.makedirs(f"{data_path}/Datasets/{data_to_conv}")

        for infile in self.files: # , outfile in zip(self.files, self.txt_files):
            # Read in the file
            with open(infile, 'r') as file:
                filedata = file.read()
            # Replace the target string
            for word in self.delete_list:
                filedata = filedata.replace(word, "")
            # Write the file out again
            with open('temp.txt', 'w') as file:
                file.write(filedata)
            with open('temp.txt', 'r') as f:
              lines = f.readlines()
              print(lines[50:80])

            with open('temp.txt', 'w') as f:
              for line in lines:
                if ('N' not in line) and ('КонсультантПлюс' not in line) and (not line.isspace()) and (not any(word.isupper() for word in line.split()) or sum(1 for c in line if c.isupper()) <= 2):
                  f.write(line)

              # for line in lines:
              #     filedata = '\n'.join(('N' not in line) and ('КонсультантПлюс' not in line)
              #                           and (not line.isspace())
              #                           and (not any(word.isupper() for word in line.split())
              #                             or sum(1 for c in line if c.isupper()) <= 2))

            # filtered_lines = [line for line in lines if 'N' not in line and 'КонсультантПлюс' not in line
            #                   and not line.isspace() and (not any(word.isupper() for word in line.split())
            #                   or sum(1 for c in line if c.isupper()) <= 2)]

            # filedata = '\n'.join(filtered_lines)

            with open('temp.txt') as f:
                filedata = f.read()

            also_delete = {':\n': ': ', ': \n': ': ',
                            ';\n': '; ', '; \n': '; '}
            for key, value in also_delete.items():
                filedata = filedata.replace(key, value)
            paragraphs = filedata.split('\n')

            for paragraph in paragraphs:
                filename = f"{data_path}/Datasets/{data_to_conv}/{str(counter).zfill(6)}.txt"
                counter += 1
                with open(filename, 'w') as file:
                    file.write(paragraph)
        return


    def clean_split_csv(self, file_path):
        counter = 0
        for infile in self.files:
            # Open the CSV file and read
            with open(infile, 'r') as filedata:
                reader = csv.DictReader(filedata, quoting=csv.QUOTE_MINIMAL)

                if not os.path.exists(f"{data_path}/Datasets/{data_to_conv}"):
                    os.makedirs(f"{data_path}/Datasets/{data_to_conv}")
                    print('new directory made')
                # Iterate through the rows and apply the replacement to the 'text' column
                for row in reader:
                    paragraph = row['Текст'].replace('\n', ' ')
                    # print(text)
                    filename = f"{data_path}/Datasets/{data_to_conv}/{str(counter).zfill(6)}.txt"
                    counter += 1
                    with open(filename, 'w') as file:
                        file.write(paragraph)
        return


    def make_to_dataset(self, files):
        if not os.path.exists(f"{data_path}/Datasets/{data_to_conv}"):
            os.makedirs(f"{data_path}/Datasets/{data_to_conv}")
        else:
            print('If errors, try refreshing the directory or this notebook manually.')


        for file_path in [os.path.join(files, file) for file in os.listdir(files)]:
            with open(file_path, 'r', encoding='utf-8') as infile:
                text = infile.read()
            # Tokenize the text into words
            words = word_tokenize(text)

            tagged_words = []
            prev_word = None  # Keep track of the previous word
            for word in words:
                if word in self.punct_marks:
                    if prev_word is not None:
                        tagged_words.append((prev_word, self.punct_marks[word]))
                    prev_word = None  # Reset prev_word after tagging it
                else:
                    if prev_word is not None:
                        tagged_words.append((prev_word, 'O'))
                    prev_word = word  # Update prev_word for the next iteration
            # Write the tagged words to the text file
            with open(file_path, 'w', encoding='utf-8') as outfile:
                for tagged_word in tagged_words:
                    outfile.write(tagged_word[0] + '\t' + tagged_word[1] + '\n')


In [ ]:
# data_to_conv_path = f'{data_path}/Raw_data/{data_to_conv}'
debug_prep = Files_Prep(data_to_conv, files_ext)
debug_prep.process_files()


In [ ]:
# data_path = '/content/drive/MyDrive/Data_punct'
# print(os.listdir(f'{data_path}/Datasets/{data_to_conv}'))